# Fitting Proteins

In [14]:
import os
import itertools
import glob
import parmed
from simtk.openmm import app
from simtk import unit
from oeommtools import utils as oeo_utils

In [2]:
residues = ['ALA', 'ASN', 'CYS', 'GLU', 'HIS',
            'LEU', 'MET', 'PRO', 'THR', 'TYR',
            'ARG', 'ASP', 'GLN', 'GLY', 'ILE',
            'LYS', 'PHE', 'SER', 'TRP', 'VAL']

## Dipeptides

I realized in this process that we don't need a separate `leap.in` file for each "protein"
so instead I will make one master input file that will parameterize all of the molecules 

In [3]:
def make_leap_str_dipeps(pairs, directory):
    """
    Parameters
    ----------
    residues: list like 
        contains 2 of the three letter residue codes
    directory: str
        path to where the output files should be stored
    
    Returns
    -------
    leap_lines: a string for the lines for the list of 
    """
    leap_str = """
divaline = sequence {N%s C%s}
savepdb divaline %s.pdb
protein = loadpdb %s.pdb
saveamberparm protein %s.prmtop %s.inpcrd
"""
    p0, p1 = pairs
    # fill in leap.in format
    fn = "%s/%s_%s" % (directory, p0, p1)
    out = leap_str % (p0, p1, fn, fn, fn, fn)

    leap_str4 = """
divaline = sequence {NALA %s %s CALA}
savepdb divaline %s.pdb
protein = loadpdb %s.pdb
saveamberparm protein %s.prmtop %s.inpcrd
"""
    fn4 = "%s/%s_%s_tetra" % (directory, p0, p1)
    out4 = leap_str4 % (p0, p1, fn4, fn4, fn4, fn4)
    
    return out, out4

def make_leap_str_tripeps(triplets, directory):
    """
    Parameters
    ----------
    residues: list like 
        contains 3 of the three letter residue codes
    directory: str
        path to where the output files should be stored
    
    Returns
    -------
    leap_lines: a string for the lines for the list of 
    """
    leap_str = """
divaline = sequence {N%s %s C%s}
savepdb divaline %s.pdb
protein = loadpdb %s.pdb
saveamberparm protein %s.prmtop %s.inpcrd
"""
    p0, p1, p2 = triplets
    # fill in leap.in format
    fn = "%s/%s_%s_%s" % (directory, p0, p1, p2)
    out = leap_str % (p0, p1, p2, fn, fn, fn, fn)
    return out

In [4]:
# create a list of all dipeptides
di_peps = itertools.combinations(residues, 2)
tri_peps = itertools.combinations(residues, 3)

# create input file
input_leap = "leap.in"
leap_in = open(input_leap, 'w')
leap_in.write("source oldff/leaprc.ff99SB")
# loop over all pairs, making input files 
for pair in di_peps:
    out, out4 = make_leap_str_dipeps(pair, "./mol_files")
    # make the *.in file
    leap_in.write(out)
    leap_in.write(out4)

for triplets in tri_peps:
    out = make_leap_str_tripeps(triplets, "./mol_files")
    leap_in.write(out)
    
leap_in.close()
os.system('tleap -f %s' % input_leap)

256

# Load systems into openMM systems

Here is what I would like to track and my planned structure:

What information do we need:

* quantitative parameters:
    - with units and "full values"
    - rounded within 0.001? to identify when parameters should be treated the same
* atom indices which will translate into OEmols with `oeommtools`
    - by parameter type
* parmed/openmm systems to make the oemols

#### What is the best way to store this data?

How to store this data isn't necessarily obvious, but this is what I'm thinking right now:

Dictionaries by parameter type with the setup:

* keys: string of quantitative parameters rounded to assigned values with two sub dictionaries:
    - 'atom indices': dictioanry with form {mol_num/id: list of atom indice tuples}
        * I think it makes more sense to keep these as dictionaries for now so that we can also have a {mol_num/id: molecule} dictionary that can be translated into the chemper format later
    - 'parameters': set of tuples so that if there are differences in rounding to the top string we can track them down and so we can store the units that go with each component

Separate dictionary with the molecules stored with the form `{mol_num/id: { 'oemol': oemol, 'parmed': parmed_system} }`. I think for a first pass it is worth storing both objects just in case, but ultimately we only really need to the oemol after the first set of dictionaries is done.

In [78]:
class parameter_dict:
    
    def __init__(self):
        self.d = dict()
    
    def items(self):
        return self.d.items()
    
    def add_key(self, key):
        if key not in self.d:
            self.d[key] = {'atom_indices': dict(), 'parameters': set(), 'units': None}
    
    def add_atoms(self, key, mol_id, atom_tuple):
        if mol_id not in self.d[key]['atom_indices']:
            self.d[key]['atom_indices'][mol_id] = list()
        self.d[key]['atom_indices'][mol_id].append(tuple(atom_tuple))
    
    def add_param(self, key, params):
        self.add_key(key)
        new_tuple = [x._value for x in params]
        self.d[key]['parameters'].add(tuple(new_tuple))
        self.d[key]['units'] = tuple([x.unit for x in params])

In [79]:
class parameter_system(object):
    
    def __init__(self):
        self.lj_dict = parameter_dict()
        self.charge_dict = parameter_dict()
        self.bond_dict = parameter_dict()
        self.angle_dict = parameter_dict()
        self.proper_dict = parameter_dict()
        self.improper_dict = parameter_dict()
        self.proper_atoms = dict()
        self.mol_dict = dict()
    
    def add_amber_system(self, prm_file=None, inp_file=None):
        """
        Adds an amber system to the clusters of parameters stored here.
        All parameters are added to relevant dictionary and returns the system
        
        Parameters
        ----------
        prm_file: path to AMBER prmtop file
        inp_file: path to AMBER inpcrd file
    
        Returns
        -------
        sys: parmed system created from these files
        """
        parm = parmed.load_file(prm_file, inp_file)
        base = prm_file.split('.')[0]
        mol_id = base.split('/')[-1]
        self.mol_dict[mol_id] = {'parmed': parm, 
                                   'oemol': oeo_utils.openmmTop_to_oemol(parm.topology, parm.positions)}
        self.add_nonbonds(parm, mol_id)
        self.add_bonds(parm, mol_id)
        self.add_angles(parm, mol_id)
        self.add_torsions(parm, mol_id)
        if len(parm.impropers) > 0:
            print("IMPRPPER")
        return parm
    
    def add_molecule(self, sys, mol_id):
        self.mol_dict[mol_id] = {'parmed': sys, 
                                   'oemol': oeo_utils.openmmTop_to_oemol(sys.topology, sys.positions)}
        
    def add_parmed_system(self, sys):
        base = sys.name.split('.')[0]
        mol_id = base.split('/')[-1]
        if mol_id not in self.mol_dict:
            self.add_molecule(sys, mol_id)
        self.add_nonbonds(sys, mol_id)
        self.add_bonds(sys, mol_id)
        self.add_angles(sys, mol_id)
        self.add_torsions(sys, mol_id)
        
    def add_nonbonds(self, sys, mol_id):
        """
        Cluster atoms based on their partial charge

        Parameters
        ----------
        sys: list like of parmed system
        charge_dict: dictionary to store data that will be updated in this function
        lj_dict: dictionary to store LJ parameters for this molecule
        mol_id: key for this system to store data in the dictionaries

        Returns
        -------
        clusters: dictionary with the form
                  {string parameter: {'atom_idices': {}}
        """    
        if mol_id not in self.mol_dict:
            self.add_molecule(sys, mol_id)
            
        for a in sys.atoms:
            # Update charge dictionary:
            charge_str = "%.3f" % a.charge
            charge_param = [a.ucharge]
            self.charge_dict.add_param(charge_str, charge_param)
            self.charge_dict.add_atoms(charge_str, mol_id, [a.idx])

            # Update LJ dictionary
            lj_str = "%.3f\t%.3f" % (a.epsilon, a.rmin)
            lj_params = [a.uepsilon, a.urmin]
            self.lj_dict.add_param(lj_str, lj_params)
            self.lj_dict.add_atoms(lj_str, mol_id, [a.idx])

    def add_bonds(self, sys, mol_id):
        if mol_id not in self.mol_dict:
            self.add_molecule(sys, mol_id)
        for b in sys.bonds:
            bond_str = "%.3f\t%.3f" % (b.type.k, b.type.req)
            bond_params = [b.type.uk, b.type.ureq]
            self.bond_dict.add_param(bond_str, bond_params)
            self.bond_dict.add_atoms(bond_str, mol_id, [b.atom1.idx, b.atom2.idx])
        
    def add_angles(self, sys, mol_id):
        if mol_id not in self.mol_dict:
            self.add_molecule(sys, mol_id)
        for an in sys.angles:
            angle_str = "%.3f\t%.3f" % (an.type.k, an.type.theteq)
            angle_params = [an.type.uk, an.type.utheteq]
            self.angle_dict.add_param(angle_str, angle_params)
            self.angle_dict.add_atoms(angle_str, mol_id, [an.atom1.idx, an.atom2.idx, an.atom3.idx])
    
    def add_torsions(self, sys, mol_id):
        if mol_id not in self.mol_dict:
            self.add_molecule(sys, mol_id)
        temp_dict = dict()
        for d in sys.dihedrals:
            if d.improper:
                imp_str = "%.3f\t%.3f\t%.3f" % (d.type.phi_k, d.type.phase, d.type.per)
                imp_params = [d.type.uphi_k, d.type.uphase, unit.Quantity(d.type.per)]
                self.improper_dict.add_param(imp_str, imp_params)
                self.improper_dict.add_atoms(imp_str, 'temp_mol', [d.atom1.idx, d.atom3.idx, d.atom2.idx, d.atom4.idx])
            else:
                atoms = tuple([d.atom1.idx, d.atom2.idx, d.atom3.idx, d.atom4.idx])
                params = (d.type.uphi_k, d.type.uphase, unit.Quantity(d.type.per))
                if atoms not in temp_dict:
                    temp_dict[atoms] = list()
                temp_dict[atoms].append(params)

        for atoms, param_list in temp_dict.items():
            new_params = [p for t in param_list for p in t]
            prop_str = '\t'.join(['%.3f' % p._value for p in new_params])
            prop_dict.add_param(prop_str, new_params)
            prop_dict.add_atoms(prop_str, 'temp_mol', atoms)


In [80]:
cwd = os.getcwd()
tetras = glob.glob('%s/mol_files/*tetra.pdb' % cwd)

store_data = parameter_system()

for pdb in tetras:
    base = pdb.split('.')[0]
    mol_id = base.split('/')[-1]
    prmFile = os.path.join('%s.prmtop' % base)
    inpFile = os.path.join('%s.inpcrd' % base)
    parm = store_data.add_amber_system(prmFile, inpFile)
    if len(parm.impropers) > 0:
        print("IMPROPERS!!!!")
        save_parm = parm
    #get_nonbond_clusters(parm, mol_id, charge_dict, lj_dict)
    #get_angle_clusters(parm, mol_id, angle_dict)
    #print(len(lj_dict.d), len(charge_dict.d), len(parm.atoms), len(angle_dict.d))

# Try making SMIRKS for charges

In [85]:
idx_list = list()
mol_list = list()
param_list = list()
for idx, me in store_data.mol_dict.items():
    idx_list.append(idx)
    mol_list.append(me['oemol'])
for label, entry in store_data.charge_dict.items():
    atom_list = list()
    for idx in idx_list:
        if idx in entry['atom_indices']:
            atom_list.append(entry['atom_indices'][idx])
        else:
            atom_list.append(list())
    param_list.append((label, atom_list))

In [88]:
from chemper.smirksify import SMIRKSifier

In [90]:
smir = SMIRKSifier(mol_list, param_list)


 Label                | SMIRKS 
 zz_-0.264            | [#6H1X4x0!r+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#6H0X3x0!r+0A](-;!@[#7H1X3x0!r+0A](-;!@[#1H0X1x0!r+0A])-;!@[#6H1,#6H2;!r;+0;X4;x0;A](-;!@[#1H0X1,#6H1X4,#6H2X4,#6H3X4;!r;+0;x0;A])(-;!@[#1H0X1x0!r+0A])-;!@[#6H0X3x0!r+0A](-;!@[#7H1X3,#8H0X1;!r;+0;x0;A])-;!@[#8H0X1x0!r+0A])-;!@[#8H0X1x0!r+0A])(-;!@[#6H2X4x0!r+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;!@[#6H2X4x0!r+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;!@[#6H2X4x0!r+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#1H0X1x0!r+0A])-;!@[#7H1X3x0!r+0A](-;!@[#1H0X1x0!r+0A])-;!@[#6H0X3x0!r+0A])-;!@[#7H1X3x0!r+0A](-;!@[#1H0X1x0!r+0A])-;!@[#6H0X3x0!r+0A](-;!@[#6H1X4x0!r+0A](-;!@[#1H0X1x0!r+0A])(-;!@[#6H1,#6H2,#6H3;!r;+0;X4;x0;A](-;!@[#16H1X2,#1H0X1,#8H1X2;!r;+0;x0;A])(-;!@[#1H0X1,#6H0X3,#6H1X4,#6H2X4,#6H3X4;!r;+0;x0;A])-;!@[#1H0X1x0!r+0A])-;!@[#7H0X3,#7H1X3,#7H3X4;!r;+0;x0;A](-;!@[#1X1,#6X3;!r;+0;H0;x0;A])-;!@[#1H0X1,#6H2X4;!r;+0;x0;A])-;!@[#8H0X1x0!r+0A] 
---------------------------------------------

ClusteringError: 
                      SMIRKSifier was not able to create SMIRKS for the provided
                      clusters with 5 layers. Try increasing the number of layers
                      or changing your clusters
                      